In [ ]:
# Obtain Canvas access token from .env file

import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
token = os.environ.get("TOKEN")

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
PROVIDER = 'swinburne'

# Overhead

In [ ]:
from requests.exceptions import HTTPError
from datetime import datetime
from tzlocal import get_localzone

def parse_iso_timestamps(data : pd.DataFrame) -> pd.DataFrame:
    """
    Replace all string timestamps in a DataFrame with datetime objects.
    All timestamps must comply with ISO-8601.

    Args:
        data (DataFrame): DataFrame containing ISO-8601 string timestamps.

    Returns:
        data (DataFrame): DataFrame containing datetime timestamps.
    """

    # Obtain all columns which can be converted
    # into a timestamp through ugly brute-forcing
    timestamp_columns = []
    for key, value in data.iloc[0].to_dict().items():
        if type(value) is str:
            try:
                datetime.fromisoformat(value)
                timestamp_columns.append(key)
            except:
                pass

    def isotime_to_timestamp(value : str | None, use_local_timezone : bool = True, as_string : bool = False):
        if type(value) is not str: return None
        
        time = datetime.fromisoformat(value)
        if use_local_timezone:
            time = time.astimezone(get_localzone())

        if as_string:
            time = time.strftime("%A, %d %B %Y, %I:%M %p")
            
        return time
    
    for column in timestamp_columns:
        data[column] = data[column].map(isotime_to_timestamp)

    return data

def process_canvas_dataframe(response : pd.DataFrame) -> pd.DataFrame:
    response = parse_iso_timestamps(response)
    return response
    
def decode_canvas_response(response : requests.Response) -> pd.DataFrame:
    """
    Converts raw Canvas API response into a DataFrame for downstream use,
    or raises Exception if the response is invalid.

    Args:
        response (Response): The raw API response obtained from requests.

    Returns:
        response_data (DataFrame): The response data.
    """
    
    if response.ok:
    
        # Canvas API will always respond in JSON format,
        # but we obtain the response in bytes, so it
        # must be decoded into a raw string and then
        # encoded into a JSON object.
        
        response_data = response.content.decode('utf-8')
        
        response_data = json.loads(response_data)

        if not response_data:
            # Empty response
            return pd.DataFrame([])
        
        if type(response_data) is not list: response_data = [response_data]
        
        response_data = pd.DataFrame(response_data)

        response_data = process_canvas_dataframe(response_data)
    
    else:
        response.raise_for_status()

    return response_data

# Get current units

In [ ]:
# Two enrolment states:
# active   - Visible on home menu
# complete - Invisible

units = requests.get(f"https://swinburne.instructure.com/api/v1/courses/", params={
    "access_token":token,
    "enrollment_state":"complete",
    "per_page":100,
})
units = decode_canvas_response(units)

In [ ]:
# Internally, Swinburne Organisation (ORG) units are assigned to enrollment term ID 1
# Since we're only concerned with academic units, let's filter out organisation units
units = units.loc[units.enrollment_term_id != 1]

# Get assignments for current units

In [ ]:
# # Bucket (Status)
# # UPCOMING:  unsubmitted
# # OVERDUE:   overdue
# # COMPLETE:  past

# assignments = []

# for unit_id in units["id"].to_list():
    
#     response = requests.get(f"https://swinburne.instructure.com/api/v1/courses/{unit_id}/assignments", params={
#         "access_token":token,
#         "order_by":"due_at",
#         "per_page":100, # Max items per unit
#         "bucket":"unsubmitted",
#     })
    
#     response = decode_canvas_response(response)
#     assignments.append(response)

# assignments = pd.concat(assignments).reset_index(drop=True)

In [ ]:
# response = requests.get(f"https://swinburne.instructure.com/api/v1/courses/56983/assignments", params={
#     "access_token":token,
#     "per_page":100, # Max items
#     "order_by":"due_at" # Prioritise urgency
# })
# assignments = decode_canvas_response(response)

# Get assignment groups for current units

In [ ]:
assignment_groups = []

for unit_id in units["id"].to_list():
    
    response = requests.get(f"https://swinburne.instructure.com/api/v1/courses/{unit_id}/assignment_groups", params={
        "access_token":token,
        "per_page":100, # Max items per unit
        "include":"assignments",
    })
    
    response = decode_canvas_response(response)
    assignment_groups.append(response)

assignment_groups = pd.concat(assignment_groups).reset_index(drop=True)

In [ ]:
# Extract assignments from groups
assignments = pd.json_normalize(assignment_groups.assignments.explode())
assignments = assignments.dropna(subset="id").reset_index(drop=True)
assignments.id = assignments.id.astype("int")
assignments = process_canvas_dataframe(assignments)

## Calculate grade contribution for each assignment

In [ ]:
# Count the total amount of points possible for each assignment group
# and add it as a feature for the data

import numpy as np
split = assignment_groups.explode("assignments")

entries = pd.json_normalize(split["assignments"])
entries = entries.add_prefix("assignment_")

split["assignment_id"] = split.assignments.map(lambda x: x['id'] if type(x) is dict else None)
split = split.dropna(subset="assignment_id")
split["assignment_id"] = split["assignment_id"].astype("int")

split = split.merge(entries, how="inner", left_on="assignment_id", right_on="assignment_id")
del split["assignments"]

total_points_possible = split.groupby("id")["assignment_points_possible"].sum().reset_index()
total_points_possible = total_points_possible.rename(columns={"assignment_points_possible" : "total_points_possible"})
assignment_groups = assignment_groups.merge(total_points_possible, how="outer", left_on="id", right_on="id")
split = split.merge(total_points_possible, how="outer", left_on="id", right_on="id")

In [ ]:
def get_assignment_group_feature(assignment_id, group_feature_name : str, assignment_groups_split : pd.DataFrame = split):
    try:
        match = assignment_groups_split.loc[assignment_groups_split.assignment_id == assignment_id]
        feature = match[group_feature_name].item()
        return feature
    except: return None

assignments["points_total_for_group"] = assignments["id"].map(lambda x: get_assignment_group_feature(x, "total_points_possible"))
assignments["group_weight"] = assignments["id"].map(lambda x: get_assignment_group_feature(x, "group_weight"))

In [ ]:
assignments["score"] = (assignments["points_possible"] / assignments["points_total_for_group"]) * (assignments["group_weight"] / 100)

In [ ]:
assignments[["name", "course_id", "score"]].sort_values("score", ascending=False)

## Get course name for each assignment from course ID

This would be so much more efficient in SQL. It's not even a contest.

In [ ]:
def get_course_name(course_id, units = units):
    if course_id not in units.id.unique(): return None

    name = units.loc[units.id == course_id].name.item()

    return name

assignments["course_name"] = assignments.course_id.map( get_course_name )

In [ ]:
assignments[["id", "course_name", "name", "due_at", "points_possible"]]